# Lab | Random Forests

### <font color='grey'> Cristian Valeria </font>
<br>
<br>

### 1.Apply the Random Forests algorithm but this time only by upscaling the data.



#### 1.1. Read

In [1]:
import pandas as pd

In [ ]:
!ls files_for_lab

In [2]:
data_target = pd.read_csv('files_for_lab/target.csv')
data_target['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [3]:
data_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TARGET_B  95412 non-null  int64  
 1   TARGET_D  95412 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.5 MB


In [4]:
data_target.shape

(95412, 2)

In [5]:
data_categorical = pd.read_csv('files_for_lab/categorical.csv')
data_categorical.shape

(95412, 22)

In [6]:
data_numerical = pd.read_csv('files_for_lab/numerical.csv')
data_numerical.shape

(95412, 315)

In [7]:
data = pd.concat([data_target, data_numerical, data_categorical], axis=1) 
data.head()

,TARGET_B,TARGET_D,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,...,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,0,0.0,0,60.000000,5,9,0,0,39,34,...,37,12,92,8,94,2,95,12,89,11
1,0,0.0,1,46.000000,6,9,16,0,15,55,...,52,2,93,10,95,12,95,12,93,10
2,0,0.0,1,61.611649,3,1,2,0,20,29,...,0,2,91,11,92,7,95,12,90,1
3,0,0.0,0,70.000000,1,4,2,0,23,14,...,28,1,87,11,94,11,95,12,87,2
4,0,0.0,0,78.000000,3,2,60,1,28,9,...,20,1,93,10,96,1,96,1,79,3


In [8]:
X = data.drop(['TARGET_B'], axis=1)
y = data['TARGET_B']

#### 1.2. Class Imbalance

In [9]:
display('y:', y.value_counts())

'y:'

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [10]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(sampling_strategy="not majority") 
X_res, y_res = ros.fit_resample(X, y)

In [11]:

display('y_res:', y_res.value_counts())

'y_res:'

0    90569
1    90569
Name: TARGET_B, dtype: int64

In [12]:
X_res = X_res.select_dtypes(include='number') #df.select_dtypes(include=numerics)

#### 1.3. RandomForest

In [13]:
from sklearn.model_selection import train_test_split

X = X_res
y = y_res

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

X_train.columns = X.columns
X_test.columns = X.columns

In [44]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10)

In [17]:
print("The R2 of the model in the TRAIN set is: {:.2f}".format(rf.score(X_train,y_train)))
print("The R2 of the model in the TEST  set is: {:.2f}".format(rf.score(X_test,y_test)))

The R2 of the model in the TRAIN set is: 1.00
The R2 of the model in the TEST  set is: 1.00


the model is overffited


### 2.Discuss the output and its impact in the bussiness scenario. Is the cost of a false positive equals to the cost of the false negative?



It depends on every case, specially on the <font color = 'blue'> marketing cost </font> related to the donation and the <font color = 'blue'> donation </font> amount mean per capita (per user). If the donation are relatively big, much bigger than the marketing cost, the false negative would impact much more than a false postive, in other words, i will not be doing the right campain to people that can be donating big amounts. In the other case if the cost of marketing related to donations is big, and donations are not so big, the cost of false positive will be bigger. 

### 3.Sklearn classification models are trained to maximize the accuracy. However, another error metric will be more relevant here. Please checkout make_scorer alogside with GridSearchCV in order to train the model to maximize the error metrics of interest in this case.


In [20]:
import numpy as np

In [32]:
n_estimators = [int(x) for x in range(5,65,5)]

max_features = ['auto', 'sqrt']

max_depth = [2,4]

min_samples_split = [2, 5]

min_samples_leaf = [1, 2]

bootstrap = [True, False]

In [35]:
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid)

{'n_estimators': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [36]:
from sklearn.model_selection import GridSearchCV
rf_Grid = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)


In [37]:
rf_Grid.fit(X_train, y_train)

Fitting 3 folds for each of 384 candidates, totalling 1152 fits
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=5; total time=   2.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   1.6s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=20; total time=   2.2s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   2.7s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=30; total time=   3.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=40; total time=   4.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=45; total time=   4.2s

[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   2.5s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=15; total time=   1.9s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=20; total time=   2.3s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=30; total time=   3.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=35; total time=   3.5s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=40; total time=   3.9s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   5.1s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_s

[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=5; total time=   2.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   1.6s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=15; total time=   1.9s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   2.7s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=30; total time=   3.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=35; total time=   3.4s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=45; total time=   4.3s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_sa

[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=5; total time=   2.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=15; total time=   2.1s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=20; total time=   2.3s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   2.6s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=35; total time=   3.5s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=40; total time=   3.9s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=45; total time=   4.4s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_sa

GridSearchCV(cv=3, estimator=RandomForestClassifier(n_estimators=10), n_jobs=4,
             param_grid={'bootstrap': [True, False], 'max_depth': [2, 4],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50,
                                          55, 60]},
             verbose=2)

In [38]:
rf_Grid.best_params_

{'bootstrap': False,
 'max_depth': 2,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 5}

In [42]:
print (f'Train Accuracy - : {rf_Grid.score(X_train,y_train):.3f}')
print (f'Test Accuracy - : {rf_Grid.score(X_test,y_test):.3f}')

Train Accuracy - : 0.569
Test Accuracy - : 0.570


Model seems less accurated than Random Forest. 